In [64]:
import dash
from dash import Dash, dcc, html, callback, Input, Output, dash_table
import plotly.express as px
import dash_bootstrap_components as dbc
import pandas as pd
import dash_mantine_components as dmc
import json
#external_stylesheet = dbc.themes.CERULEAN
subject_viewer_app = Dash(__name__, external_stylesheets=[dbc.themes.CERULEAN]) #no use pages for inline jupyter dev

data=pd.read_excel('/Users/demo/Library/CloudStorage/Box-Box/Holmes_Lab_Wiki/PCX_Round2/Data/Subject_tracker_PCR.xlsx', sheet_name='clean_data')
subject_ids = data['Qual ID'].unique()

clinRatings=pd.read_csv('/Users/demo/Library/CloudStorage/Box-Box/Holmes_Lab_Wiki/PCX_Round2/Data/behavioral/PCX_ClinicalVisit_ClinicianRatings/PCX_ClinicalVisit_ClinicianRatings_June 18, 2025_17.59.csv')
supp=pd.read_csv('/Users/demo/Library/CloudStorage/Box-Box/Holmes_Lab_Wiki/PCX_Round2/Data/behavioral/PCX_SupplementalBattery/PCX_SupplementalBattery_June 18, 2025_17.12.csv')
fmriBattery=pd.read_csv('/Users/demo/Library/CloudStorage/Box-Box/Holmes_Lab_Wiki/PCX_Round2/Data/behavioral/PCX_fMRIVisit_SelfReport/PCX_fMRIVisit_SelfReport_June 18, 2025_17.13.csv')
clinBattery=pd.read_csv('/Users/demo/Library/CloudStorage/Box-Box/Holmes_Lab_Wiki/PCX_Round2/Data/behavioral/PCX_ClinicalVisit_ClinicianRatings/PCX_ClinicalVisit_ClinicianRatings_June 18, 2025_17.59.csv')

survey_to_df = {
	'panss': clinRatings, 
	'madrs': clinRatings,
	'bprs': clinRatings,
	'ymrs': clinRatings,
	'cssrs': clinRatings,
	'fam_conditions': fmriBattery,
	'asi': fmriBattery,
	'dass': fmriBattery,
	'stai': clinBattery,
	'neoffi': clinBattery,
	'bapq': clinBattery,
	'qids': clinBattery,
}

def create_sub(df, subject):
	sub_df=df[df['Qual ID']==subject].copy()
	sub_df=sub_df.reset_index()
	sub_str=sub_df.to_json()
	return sub_df, sub_str


def filter_subject_qualtrics(df, subject):
	return df[df['SUBJECT_ID']==subject]

def render_table(subject, survey_name=None, survey_regex=None):
	if survey_name is not None:
		df = survey_to_df[survey_name]
		cols = [col for col in df.columns if survey_name in col and 'notes' not in col and 'total' not in col]
		if len(cols)==0:
			return html.Div(children=[f'No data for subject {subject}, survey {survey_name}'])
		else:
			survey_df = df[['SUBJECT_ID']+cols]
			mapping = dict(zip(survey_df.columns, survey_df.iloc[0]))
			sub_df = filter_subject_qualtrics(survey_df, subject)
			long_df = sub_df.melt(id_vars='SUBJECT_ID', var_name='name', value_name='value')
			long_df['label'] = long_df['name'].map(mapping)
			long_df = long_df[['label', 'value']]
			long_df['label'] = long_df['label'].str.split('\n').str[0]
			long_df = long_df.sort_values('value', ascending=False)
		
		return dash_table.DataTable(
			data=long_df.to_dict('records'),
			columns=[{'id': c, 'name': c} for c in long_df.columns],
			css=[{
				'selector': '.dash-spreadsheet td div',
				'rule': '''
					line-height: 15px;
					max-height: 30px; min-height: 30px; height: 30px;
					display: block;
					overflow-y: hidden;
				'''
			}],
			tooltip_data=[
				{
					column: {'value': str(value), 'type': 'markdown'}
					for column, value in row.items()
				} for row in df.to_dict('records')
			],
			tooltip_duration=None,
			style_cell={'textAlign': 'left'})      
	else:
		return None

def render_chart(subject, survey_name=None, survey_regex=None):
	if survey_name is not None:
		df = survey_to_df[survey_name]
		cols = [col for col in df.columns if survey_name in col and 'notes' not in col and 'total' not in col]
		if len(cols)==0:
			return html.Div(children=[f'No data for subject {subject}, survey {survey_name}'])
		else:
			survey_df = df[['SUBJECT_ID']+cols]
			mapping = dict(zip(survey_df.columns, survey_df.iloc[0]))
			sub_df = filter_subject_qualtrics(survey_df, subject)
			long_df = sub_df.melt(id_vars='SUBJECT_ID', var_name='name', value_name='value')
			long_df['label'] = long_df['name'].map(mapping)
			long_df = long_df[['label', 'value']]
			long_df['label'] = long_df['label'].str.split('\n').str[0]
			long_df['label'] = long_df['label'].str.lower()
			long_df['value'] = pd.to_numeric(long_df['value'], errors='coerce')
			fig = px.bar(long_df, x='label', y='value', text_auto='.2s', range_y=[0,10])
			fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
			fig.update_layout(xaxis_tickangle=-35)  
			return dcc.Graph(
				figure=fig,
				config={'displayModeBar': True}, 
				style={'width': '100%',  'height': '100%' },  className = "outer-graph",
			)


def render_graph(subject):
		full_df = pd.DataFrame()
		for df in [clinRatings, clinBattery, supp, fmriBattery]:
			df.columns = df.iloc[0]
			sub_df = filter_subject_qualtrics(df, subject)
			full_df = pd.concat([full_df, sub_df], axis=1)
		cols = [col for col in full_df.columns if any(item in col for item in list(survey_to_df.keys())) in col and 'notes' not in col and 'total' not in col]
		if len(cols)==0:
			return html.Div(children=[f'No data for subject {subject}'])
		else:
			long_df = sub_df.melt(id_vars='SUBJECT_ID', var_name='name', value_name='value')
			long_df = long_df[['label', 'value']]
			for survey in survey_to_df.keys():
				long_df.loc[long_df['label'].str.contains(survey), 'label'] = long_df.loc[long_df['name'].str.contains(survey), 'name']
			long_df['label'] = long_df['label'].str.split('\n').str[0]
			long_df['label'] = long_df['label'].str.lower()
			long_df['value'] = pd.to_numeric(long_df['value'], errors='coerce')
			fig = px.line(long_df, x='label', y='value', color='', text_auto='.2s', range_y=[0,10])
			fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
			fig.update_layout(xaxis_tickangle=-35)  
			return dcc.Graph(
				figure=fig,
				config={'displayModeBar': True}, 
				style={'width': '100%',  'height': '100%' },  className = "outer-graph",
			)

subject_viewer_app.layout = html.Div([
	dcc.Location(id="url"),
	html.H1(children='Subject Viewer', style={'margin':20}),
	html.Div(children=[
		html.Div(children=[
			html.Div("Use this dropdown to select the subject"),
			dcc.Dropdown(subject_ids, id='subject-picker',clearable=False, value='qualr200'),
			dcc.Store(id='subject-id'),
			dcc.Markdown(id='caption'),
		], style={'width': '48%', 'float': 'left', 'display': 'inline-block'}),
		html.Div(children=[
			dcc.RadioItems(list(survey_to_df.keys()), id='survey', value='panss'),
			dcc.Tabs(id="content-type", value='Chart', children=[
				dcc.Tab(label='Chart', value='Chart'),
				dcc.Tab(label='Table', value='Table'),
			]),
			html.Div(id='tabs-content')
		], style={'width': '48%', 'right': '', 'display': 'inline-block'}),
	], style={'display': 'flex', 'margin':20, 'flexDirection': 'row'}
	),
])

@callback(
	Output(component_id='subject-id', component_property='data'),
	Input(component_id='subject-picker', component_property='value')
)
def update_subject(subject):
	return subject


@callback(
	Output(component_id='caption', component_property='children'), 
	Input(component_id='subject-id', component_property='data')
)
def update_caption(subject):
	markdown_text = '''
	### Your subject is {}
	'''.format(subject)
	return markdown_text

@callback(
		Output('tabs-content', 'children'),
		Input('survey', 'value'),
		Input('content-type', 'value'),
		Input('subject-id', 'data'),
)
def render_content(survey, contentType, subject):
	if contentType=='Table':
		return render_table(subject, survey_name=survey)
	if contentType=='Chart':
		return render_chart(subject, survey_name=survey)

if __name__ == '__main__':
	subject_viewer_app.run(debug=True, jupyter_mode="inline", port=8052)
	


In [ ]:

clinRatings=pd.read_csv('/Users/demo/Library/CloudStorage/Box-Box/Holmes_Lab_Wiki/PCX_Round2/Data/behavioral/PCX_ClinicalVisit_ClinicianRatings/PCX_ClinicalVisit_ClinicianRatings_June 18, 2025_17.59.csv')
supp=pd.read_csv('/Users/demo/Library/CloudStorage/Box-Box/Holmes_Lab_Wiki/PCX_Round2/Data/behavioral/PCX_SupplementalBattery/PCX_SupplementalBattery_June 18, 2025_17.12.csv')
fmriBattery=pd.read_csv('/Users/demo/Library/CloudStorage/Box-Box/Holmes_Lab_Wiki/PCX_Round2/Data/behavioral/PCX_fMRIVisit_SelfReport/PCX_fMRIVisit_SelfReport_June 18, 2025_17.13.csv')
clinBattery=pd.read_csv('/Users/demo/Library/CloudStorage/Box-Box/Holmes_Lab_Wiki/PCX_Round2/Data/behavioral/PCX_ClinicalVisit_ClinicianRatings/PCX_ClinicalVisit_ClinicianRatings_June 18, 2025_17.59.csv')

survey_to_df = {
	'panss': clinRatings, 
	'madrs': clinRatings,
	'bprs': clinRatings,
	'ymrs': clinRatings,
	'cssrs': clinRatings,
	'fam_conditions': fmriBattery,
	'asi': fmriBattery,
	'dass': fmriBattery,
	'stai': clinBattery,
	'neoffi': clinBattery,
	'bapq': clinBattery,
	'qids': clinBattery,
}

# for df in [clinRatings, clinBattery, supp, fmriBattery]:
# 	if 'SUBJECT_ID' in df.columns:
# 		print('yes')
# 	subject='qualr200'
# 	sub_df = filter_subject_qualtrics(df, subject)
# 	display(sub_df)

subject='qualr200'
full_df = pd.DataFrame()
for df in [clinRatings, clinBattery, supp, fmriBattery]:
	subject='qualr200'
	sub_df = filter_subject_qualtrics(df, subject)
	full_df = pd.concat([full_df, sub_df], axis=1)

print('full_df')
display(full_df['SUBJECT_ID'])

cols = [col for col in full_df.columns if any(item in col for item in list(survey_to_df.keys())) and 'notes' not in col and 'total' not in col] 


sub_df = full_df[['SUBJECT_ID']+cols]
# make only one SUBJECT_ID column
# fix attribute error from sub_df.melt 
long_df = sub_df.melt(id_vars='SUBJECT_ID', var_name='name', value_name='value')
long_df = long_df[['label', 'value']]
for survey in survey_to_df.keys():
	long_df.loc[long_df['label'].str.contains(survey), 'label'] = long_df.loc[long_df['name'].str.contains(survey), 'name']
long_df['label'] = long_df['label'].str.split('\n').str[0]
long_df['label'] = long_df['label'].str.lower()
long_df['value'] = pd.to_numeric(long_df['value'], errors='coerce')

long_df = render_graph('qualr200')
display(long_df)
# fig = px.line(long_df, x='label', y='value', color='', text_auto='.2s', range_y=[0,10])
# fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
# fig.update_layout(xaxis_tickangle=-35)  
# fig.show()



full_df


,SUBJECT_ID,SUBJECT_ID,SUBJECT_ID,SUBJECT_ID
2,qualr200,qualr200,qualr200,qualr200


AttributeError: 'DataFrame' object has no attribute 'dtype'

In [ ]:

# subject='qualr200'
# sub_json=df[df['Qual ID']==subject]
# display(sub_json)

# subject='qualr201'
# sub_json=df[df['Qual ID']==subject]
# display(sub_json)

# subject='qualr205'
# sub_json=df[df['Qual ID']==subject]
# display(sub_json)

def render_chip(stage, stage_chips):
	variant = stage_chips[stage]['variant']
	color = stage_chips[stage]['color']
	text=stage_chips[stage]['text']

	dmc.Chip(
		text,
		variant="light",
		color="#5c7cfa",
		size="sm",
		radius="lg",
		# other props...
	)

# get from dcc store
def check_notification(subject, last_completed_stage, last_record):
	if last_completed_stage == 'Clinical Interview Session Time':
		notification=f'Send MRI Screener email to {subject}!'
		type='to do'
	if last_completed_stage == 'Sent MRI Screener email?':
		notification=f'Check MRI Screener for {subject}- sent on {last_record}, if not done, check in.'
		type='to do'

	return notification, type


def create_sub_json(df, subject):
	sub_df=df[df['Qual ID']==subject].copy()
	sub_df=sub_df.reset_index()
	sub_json_str=sub_df.to_json()
	return sub_json_str


def check_stage(sub_json_str):
	sub_json = json.loads(sub_json_str)
	next_key=[]

	# Go through each key, find the first key where value = NA, set that as 'next key'
	for key, value in sub_json.items():
		if not pd.isna(value['0']):
			last_completed_stage=key
			last_record=value['0']
		elif pd.isna(value['0']) and next_key==99:
			next_key=key
			break

			
	if next_key==99: 
		final_stage=last_completed_stage
		return sub_json, final_stage, last_completed_stage, last_record
	else:
		return sub_json, next_key, last_completed_stage, last_record


data=pd.read_excel('/Users/demo/Library/CloudStorage/Box-Box/Holmes_Lab_Wiki/PCX_Round2/Data/Subject_tracker_PCR.xlsx', sheet_name='clean_data')
# sub_str=create_sub(tracker, 'qualr201')
# sub_json, stage, last_completed_stage, last_record = check_stage(sub_str)
# print(f'for 201, stage={stage},last_completed={last_completed_stage}: {last_record}')
sub_df, sub_str=create_sub(data, 'qualr204')
print(sub_str)
sub_json, stage, last_completed_stage, last_record = check_stage(sub_str)
print(f'for 202, stage={stage},last_completed={last_completed_stage}: {last_record}')




{"index":{"0":4},"Qual ID":{"0":"qualr204"},"PCR ID":{"0":"PCR204"},"Clinical Interview Session Date":{"0":45821},"Clinical Interview Session Time":{"0":"5:35PM"},"Sent MRI Screener email?":{"0":"Yes 6\/16\/25"},"Unnamed: 5":{"0":45824},"MRI Eligible?":{"0":0},"MRI Eligibility Notes":{"0":0},"Sent MRI Scheduling Email?":{"0":"Yes"},"MRI Scheduling notes":{"0":null},"SESSION DATE":{"0":45833},"SESSION TIME":{"0":"1:45pm-5pm"},"SCAN TIME":{"0":"2:15-3:45"},"Need uber? If so, pickup\/dropoff location":{"0":0},"Uber receipt submitted?":{"0":null},"Consent to study?":{"0":null},"Consent to NDA release?":{"0":null},"Consent to MindLAMP?":{"0":null},"MRI Session self-report battery":{"0":null},"MRI scan":{"0":null},"MRI Scan Notes":{"0":null},"Movie Rater Task":{"0":null},"Session Notes (anything missing, MRI notes)":{"0":null},"Sent post-session email? (With mindLAMP  + supplemental survey)":{"0":null},".":{"0":null},"phone type":{"0":null},"qualtrics_id":{"0":null},"MindLAMP_username (this 

In [ ]:

def check_mri(tracker):
	for subject in tracker['PCR ID'].unique():
	session_date=sub_df.mask()
	mri_done=sub_df.loc[0, 'MRI scan']
	if not pd.isna(session_date) and pd.isna(mri_done):
		return session_date
	
tracker=pd.read_excel('/Users/demo/Documents/Subject_tracker_PCR.xlsx', sheet_name='MRI Session')
subject='qualr202'
sub_df=df[df['Qual ID']==subject]
print(sub_df.columns.to_list())
display(sub_df[['Session Date','MRI scan']])
print(sub_df.shape)
session_date=sub_df.loc[1, 'Session Date']
mri_done=sub_df.loc[1, 'MRI scan']
print(type(session_date))
print(session_date)
print(mri_done)
# date=check_mri(subject, sub_df)


{"index":{"0":4},"Qual ID":{"0":"qualr204"},"PCR ID":{"0":"PCR204"},"Column1":{"0":null},"Clinical Interview Session Date":{"0":1749772800000},"Clinical Interview Session Time":{"0":"5:35PM"},"Sent MRI Screener email?":{"0":"Yes 6\/16\/25"},"MRI Screener Done Date?":{"0":1750032000000},"MRI Eligible?":{"0":null},"MRI Eligibility Notes":{"0":null},"If eligibility TBD, asked Jeff?":{"0":null},"Sent MRI Scheduling Email?":{"0":"Yes"},"MRI Scheduling notes":{"0":null},"MRI Session Date":{"0":1750809600000},"Session Start (Scheduled)":{"0":"1:45pm-5pm"},"Session End (Scheduled)":{"0":"1:45pm-5pm"},"Scan Start (Scheduled)":{"0":"2:15-3:45"},"Scan End (Scheduled)":{"0":"2:15-3:45"},"Need uber? If so, pickup\/dropoff location":{"0":null},"Consent to study?":{"0":null},"Consent to NDA release?":{"0":null},"Consent to MindLAMP?":{"0":null},"qualtrics_id":{"0":"qualr204"},"MindLAMP_username (this + @lamp.com)":{"0":"qualr204@lamp.com"},"MindLAMP_password":{"0":"4vWWTO"},"Created MindLAMP User?":{

In [61]:
import os
import sys
import dash
from dash import html, dcc, callback, Input, Output
import dash
from dash import html, dcc, callback, Input, Output
import plotly 
import plotly.express as px
import pandas as pd
import xarray as xr
import importlib
import logging
import re
import pickle
import inspect

# Set project dir
project_dir = os.path.expanduser('~/Library/CloudStorage/Box-Box/Holmes_Lab_Wiki/PCX_Round2')
dashboard = Dash(__name__, external_stylesheets=[dbc.themes.CERULEAN]) #no use pages for inline jupyter dev

# # Import custom scripts
sys.path.append(project_dir)
import scripts.paths as paths
import scripts.sub_id as sub_id
if 'scripts.paths' in sys.modules:
    importlib.reload(sys.modules['scripts.paths'])
if 'scripts.sub_id' in sys.modules:
    importlib.reload(sys.modules['scripts.sub_id'])
from scripts.sub_id import extract
from scripts.paths import get_path


# # Register page into dash app as pagename
# dash.register_page(__name__, 
#     path='/dashboard', # these 3 are automatically generated like this, but you can edit them
#     title='Dashboard',
#     name='Dashboard'
# )

# Set up logging
logging.basicConfig(
    filename='dashboard.log',        # File to write logs to, saved in working directory
    filemode='a',              # 'a' for append, 'w' to overwrite each time
    level=logging.INFO,        # Minimum logging level
    format='%(asctime)s - %(levelname)s - %(filename)s:%(lineno)d - %(message)s'
)

subs_df=pd.read_excel('/Users/demo/Library/CloudStorage/Box-Box/Holmes_Lab_Wiki/PCX_Round2/Data/Subject_tracker_PCR.xlsx', sheet_name='clean_data')

# Skip displaying some irrelevant fields if necessary, change
heatmap_columns = subs_df.columns.to_list()
subs_df_binary = subs_df.fillna(0)
subs_df_filtered = subs_df_binary.loc[subs_df_binary['Clinical Interview Session Date'] != 0, :]

# # Now the structure is:
# # |PCR ID:  | Qual ID    |    diagnosis    |  mri_surveys  | <-- these are all columns in clean_data
# # |PCR200   |  qualr200  |     <value>     |      0        | ...



# App layout
dashboard.layout = html.Div([
    html.H1('Subject Tasks Completed'),
    dcc.Dropdown(
        id='subject_id',
        options=['All']+[sub_id for sub_id in subs_df_filtered['Qual ID'].unique()],
        placeholder="Select a subject",
        value='All'
    ),
    dcc.Graph(figure={}, id='dashboard-graph')
])


# Controls to filter the figure by subject ID and
@callback(
    Output('dashboard-graph', 'figure'),
    Input('subject_id', 'value')
)

def cb(subject_id):
    if subject_id == "All":
        filtered_df = subs_df_binary
    else:
        filtered_df = subs_df_binary[subs_df_binary['Qual ID'] == subject_id]
    # Binarize non-zero values
    filtered_df_bin = filtered_df.where(filtered_df == 0, 1)
    #display(filtered_df_bin)
    
    # Create the figure
    fig = px.imshow(filtered_df_bin, origin='lower',
                    title="Tasks Completed by Participants",
                    zmin=0, zmax=1, color_continuous_scale=[[0, "white"], [1, "black"]],
                    labels=dict(x="Tasks Completed", y="Subject", color="Done = Black"),
                    x=subs_df.columns,
                    y=subs_df['Qual ID'])
    return fig

if __name__ == '__main__':
	dashboard.run(debug=False, jupyter_mode="inline", port=8054)
	




In [62]:
import os
import sys
import re
import dash
from dash import html, dcc, callback, Input, Output
import plotly 
import plotly.express as px
import pandas as pd
import xarray as xr
import importlib
import logging
import pickle
import inspect

# Import custom scripts
project_dir = os.path.basename(os.getcwd())
sys.path.append(project_dir)
from scripts.update_dataframes import update_dfs
import scripts.paths as paths
import scripts.sub_id as sub_id
if 'scripts.paths' in sys.modules:
    importlib.reload(sys.modules['scripts.paths'])
if 'scripts.sub_id' in sys.modules:
    importlib.reload(sys.modules['scripts.sub_id'])


# Register page into dash app as pagename
# dash.register_page(__name__, path="/subject")

# Set PCX Project Data path
pcx_dir = os.path.expanduser("~/Library/CloudStorage/Box-Box/Holmes_Lab_Wiki/PCX_Round2")
subject_page = Dash(__name__, external_stylesheets=[dbc.themes.CERULEAN]) #no use pages for inline jupyter dev

# Update DFs
subs_df, mindlamp_df, selected_cols, readable_cols = update_dfs(pcx_dir)
power_df = mindlamp_df[mindlamp_df['sensor']=='power']
accel_df = mindlamp_df[mindlamp_df['sensor']=='accel']
gps_df = mindlamp_df[mindlamp_df['sensor']=='gps']

# App layout
subject_page.layout = html.Div([
    html.H1('Data by Subject'),
    dcc.Dropdown(id="subject_id", value='sub-PCX-PCT927', clearable=False,
        options=[{"label": f'{y}', "value": y} for y in power_df['subject_id'].unique()]),
    dcc.RadioItems(id='days', value='All days', 
        options=['All days','Weekdays','Weekends']),

    dcc.Graph(figure={}, id='phone_use-graph'),
    dcc.Graph(figure={}, id='daily_phoneuse-graph'),
    dcc.Graph(figure={}, id='activity-graph'),
    dcc.Graph(figure={}, id='gps-graph')

])

# Add controls to build the interaction
@callback(
    Output(component_id='phone_use-graph', component_property='figure'),
    Input(component_id='subject_id', component_property='value'),
    Input(component_id='days', component_property='value')
)


def cb(subject_id, days):
    """Callback to update the figure based on the selected id"""
    sub = subject_id
    sub_df = power_df.query("subject_id == @sub")
    if days == 'Weekdays':
        sub_df = sub_df[sub_df['weekday'].astype(str).str.contains('1|2|3|4|5')]
    elif days == 'Weekends':
        sub_df = sub_df[sub_df['weekday'].astype(str).str.contains('6|7')]
    xr_power = xr.DataArray(sub_df[selected_cols].values, 
                        dims=["Days", "Hours of the Day"],
                        coords={"Days": sub_df['day'], "Hours of the Day": readable_cols})
    
    fig = px.imshow(xr_power, origin='lower', title=f'Phone Activity (minutes each hour) for sub-{sub}',
                    zmin=0,zmax=60,height=400, width=600)
    return fig

# Add controls to build the interaction
@callback(
    Output(component_id='daily_phoneuse-graph', component_property='figure'),
    Input(component_id='days', component_property='value')
)


def cb(days):
    """Callback to update the figure based on the selected id"""
    if days == 'All days':
        power_df_filtered = power_df
    elif days == 'Weekdays':
        power_df_filtered = power_df[power_df['weekday'].astype(str).str.contains('1|2|3|4|5')]
    elif days == 'Weekends':
        power_df_filtered = power_df[power_df['weekday'].astype(str).str.contains('6|7')]
    
    fig = px.line(power_df_filtered, x='day', y='daily_mins', color='subject_id', title=f'Phone Activity (minutes each hour) for each subject', 
                  height=400, width=600)
    return fig


# Add controls to build the interaction
@callback(
    Output(component_id='activity-graph', component_property='figure'),
    Input(component_id='subject_id', component_property='value'),
    Input(component_id='days', component_property='value')
)

def cb(subject_id, days):
    """Callback to update the figure based on the selected id"""
    sub = subject_id
    sub_df = accel_df.query("subject_id == @sub")
    if days == 'Weekdays':
        sub_df = sub_df[sub_df['weekday'].astype(str).str.contains('1|2|3|4|5')]
    elif days == 'Weekends':
        sub_df = sub_df[sub_df['weekday'].astype(str).str.contains('6|7')]
    xr_power = xr.DataArray(sub_df[selected_cols].values, 
                        dims=["Days", "Hours of the Day"],
                        coords={"Days": sub_df['day'], "Hours of the Day": readable_cols})
    
    fig = px.imshow(xr_power, origin='lower', title=f'Physical Activity (acceleration speed) minutes each hour for sub-{sub}',
                    zmin=0,zmax=60,height=400, width=600)
    return fig



@callback(
    Output(component_id='gps-graph', component_property='figure'),
    Input(component_id='subject_id', component_property='value'),
    Input(component_id='days', component_property='value')
)


def cb(subject_id, days):
    """Callback to update the figure based on the selected id"""
    sub = subject_id
    sub_df = gps_df.query("subject_id == @sub")
    if days == 'Weekdays':
        sub_df = sub_df[sub_df['weekday'].astype(str).str.contains('1|2|3|4|5')]
    elif days == 'Weekends':
        sub_df = sub_df[sub_df['weekday'].astype(str).str.contains('6|7')]
    xr_power = xr.DataArray(sub_df[selected_cols].values, 
                        dims=["Days", "Hours of the Day"],
                        coords={"Days": sub_df['day'], "Hours of the Day": readable_cols})
    
    fig = px.imshow(xr_power, origin='lower', title=f'Mobility (distance each hour) for sub-{sub}',
                    zmin=0,zmax=60,height=400, width=600)
    return fig

"""Callback to update the figure based on the selected id"""
sub = 'sub-PCX-PCT927'
sub_df = accel_df.query("subject_id == @sub")
logging.info(sub_df.head())


if __name__ == '__main__':
	subject_page.run(debug=False, jupyter_mode="inline", port=8056)
	

TimeoutError: [Errno 60] Operation timed out